## Implementation of the Neural Network archtechture

This shows one of the possible NN architecture. Other architectures can be used as well and loaded from the models.py file. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.models import *
from src.train import *
from src.data_loader import *


In [3]:
# Load the augmented data 
# x_train, y_train, x_predict = load_base_data() # In case you want to load the base data
x_train, y_train, x_predict = load_augmented_data()

# Split the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train)

# Rescale the data and convert it to tensor
X_train, X_test, X_predict, Y_train, Y_test = rescale_data_convert_to_tensor(X_train, X_test, x_predict, Y_train, Y_test)

# Augment the data by adding gaussian noise
X_train, X_test, Y_train, Y_test = augment_data(X_train, X_test, Y_train, Y_test)

# Apply PCA and reduce the number of features to 500
X_train, X_test, X_predict = apply_PCA(X_train, X_test, X_predict, n_components=500)
print(X_train.shape)

# Create the datasets and the dataloaders
train_dataset, train_loader, test_dataset, test_loader = get_datasets(X_train, Y_train, X_test, Y_test)

(7744, 500)


In [4]:
# Set the hyperparameters for the encoder decoder training 
epochs = 3
device = "cuda" if torch.cuda.is_available() else "cpu"
neural_net = NeuralNet_1(X_train.shape[1]).to(device)

neural_net.apply(init_weights)

optimizer = torch.optim.Adam(neural_net.parameters(), lr=1e-3, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2, verbose=True, 
)

model_path = './models/neural_net.pth'

count_parameters(neural_net)

+--------------+------------+
|   Modules    | Parameters |
+--------------+------------+
| enc.0.weight |   256000   |
|  enc.0.bias  |    512     |
| enc.3.weight |   131072   |
|  enc.3.bias  |    256     |
| enc.6.weight |    256     |
|  enc.6.bias  |     1      |
+--------------+------------+
Total Trainable Params: 388097


c:\Users\jerem\anaconda3\envs\ml\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


388097

In [5]:
#training loop
train_neural_net(
    model = neural_net,
    optimizer = optimizer,
    scheduler = scheduler,
    train_loader = train_loader,
    test_loader = test_loader,
    Y_test = Y_test,
    model_path = model_path,
    epochs=epochs,
)

  0%|          | 0/968 [00:00<?, ?it/s]

  0%|          | 0/968 [00:00<?, ?it/s]

  0%|          | 0/968 [00:00<?, ?it/s]

In [6]:
# Evaluate the model locally and generate the submission file
neural_net = load_model(NeuralNet_1(X_train.shape[1]), model_path)
spearmanr_neural_net(neural_net, X_test, Y_test)
generate_submission_neural_net(X_predict, x_predict, neural_net)

c:\Users\jerem\Documents\Informatique\MLCourse\ml-project-2-the_underfitters\src\models.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


Spearmann R coefficient for neural net is : 0.605841909355867
Predicted 304 samples
